In [1]:
import numpy as np
import json
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization, merge
from keras import backend as K

Using TensorFlow backend.


In [2]:
def format_decimal(arr, places=8):
    return [round(x * 10**places) / 10**places for x in arr]

### pipeline 18

In [16]:
data_in_shape = (8, 8, 2)

input_layer_0 = Input(shape=data_in_shape)
branch_0 = Conv2D(4, 3, 3, activation='relu', border_mode='same', subsample=(1, 1), dim_ordering='tf', bias=True,
                         name='conv_0-0')(input_layer_0)

input_layer_1 = Input(shape=data_in_shape)
branch_1 = Conv2D(2, 1, 1, activation='relu', border_mode='valid', subsample=(1, 1), dim_ordering='tf', bias=True,
                         name='conv_1-0')(input_layer_1)
branch_1 = Conv2D(4, 3, 3, activation='relu', border_mode='same', subsample=(1, 1), dim_ordering='tf', bias=True,
                         name='conv_1-1')(branch_1)
branch_1 = Conv2D(2, 1, 1, activation='relu', border_mode='valid', subsample=(1, 1), dim_ordering='tf', bias=True,
                         name='conv_1-2')(branch_1)

input_layer_2 = Input(shape=data_in_shape)
branch_2 = Conv2D(5, 3, 3, activation='relu', border_mode='same', subsample=(1, 1), dim_ordering='tf', bias=True,
                         name='conv_2-0')(input_layer_2)
branch_2 = Conv2D(3, 3, 3, activation='relu', border_mode='same', subsample=(1, 1), dim_ordering='tf', bias=True,
                         name='conv_2-1')(branch_2)

output_layer = merge([branch_0, branch_1, branch_2], mode='concat')
model = Model(input=[input_layer_0, input_layer_1, input_layer_2], output=output_layer)

data_in = []
for i in range(3):
    np.random.seed(19000 + i)
    data_in.append(np.expand_dims(2 * np.random.random(data_in_shape) - 1, axis=0))

# set weights to random (use seed for reproducibility)
weights = []
for i, w in enumerate(model.get_weights()):
    np.random.seed(19000 + i)
    weights.append(2 * np.random.random(w.shape) - 1)
model.set_weights(weights)

result = model.predict(data_in)

print({
    'inputs': [{'data': format_decimal(data_in[i].ravel().tolist()), 'shape': list(data_in_shape)} for i in range(3)],
    'weights': [{'data': format_decimal(weights[i].ravel().tolist()), 'shape': list(weights[i].shape)} for i in range(len(weights))],
    'expected': {'data': format_decimal(result[0].ravel().tolist()), 'shape': list(result[0].shape)}
})

{'weights': [{'shape': [1, 1, 2, 2], 'data': [-0.71596766, 0.83019575, 0.12233745, 0.92591957]}, {'shape': [2], 'data': [-0.24747862, 0.16567801]}, {'shape': [3, 3, 2, 4], 'data': [-0.94835971, -0.09864511, -0.96668335, -0.11678862, -0.0684701, 0.35297961, 0.68186505, -0.64464683, 0.60428433, -0.12843896, -0.66738358, -0.96252509, 0.11343417, -0.26810103, 0.62478478, -0.92468562, 0.12092665, -0.03926093, -0.48419303, -0.2015424, -0.59762146, 0.48302104, 0.46979752, -0.33142447, 0.6379101, 0.29292009, -0.08235647, 0.58146891, -0.95709137, 0.91108344, -0.54017656, 0.7763715, -0.0947622, 0.44968087, -0.1750356, -0.16006343, -0.98221201, 0.16236873, 0.02533719, -0.7193739, -0.83571764, -0.09914948, -0.05324984, -0.58456162, 0.81909889, -0.49005946, -0.37903645, 0.7714574, 0.16240276, 0.77738959, -0.60000374, -0.4806184, -0.84023324, -0.36447888, 0.18264885, -0.32894268, 0.36280974, -0.85252119, 0.61026127, 0.04979669, -0.15516699, -0.67961233, -0.36392683, -0.8904356, 0.90778897, 0.9317497

In [17]:
import json
config = json.loads(model.to_json())
print([x['config']['name'] for x in config['config']['layers']])

['input_5', 'conv_1-0', 'input_6', 'input_4', 'conv_1-1', 'conv_2-0', 'conv_0-0', 'conv_1-2', 'conv_2-1', 'merge_2']
